In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
%matplotlib inline
plt.style.use('ggplot')

from simulation.analyse import get_win_probabilities, get_simulations

In [40]:
match_template = pd.read_csv('data/original/wc_2018_games_real.csv')
teams = pd.unique(match_template[['home_team', 'away_team']].values.ravel('K'))[0:32]
match_ids = match_template["id"]
match_template["outcome"] = np.sign(match_template["home_score"] - match_template["away_score"]) 
match_template

,id,away_score,away_team,date,home_score,home_team,group,outcome
0,0,0,Saudi Arabia,2018-06-14,5,Russia,A,1
1,1,1,Uruguay,2018-06-15,0,Egypt,A,-1
2,2,1,IR Iran,2018-06-15,0,Morocco,B,-1
3,3,3,Spain,2018-06-15,3,Portugal,B,0
4,4,1,Australia,2018-06-16,2,France,C,1
5,5,1,Iceland,2018-06-16,1,Argentina,D,0
6,6,1,Denmark,2018-06-16,0,Peru,C,-1
7,7,0,Nigeria,2018-06-16,2,Croatia,D,1
8,8,1,Serbia,2018-06-17,0,Costa Rica,E,-1
9,9,1,Mexico,2018-06-17,0,Germany,F,-1


In [55]:
match_bets = pd.read_csv('data/original/wc_2018_bets.csv')
match_bets.loc[(match_bets["1"] < match_bets["2"]) & ((match_bets["1"] < match_bets["X"])), "bet_model_outcome"] = 1
match_bets.loc[(match_bets["2"] < match_bets["1"]) & ((match_bets["2"] < match_bets["X"])), "bet_model_outcome"] = -1
match_bets.loc[(match_bets["X"] < match_bets["1"]) & ((match_bets["X"] < match_bets["2"])), "bet_model_outcome"] = 0
match_bets["overroundness"] = 1/match_bets["1"] + 1/match_bets["X"] + 1/match_bets["2"]
match_bets["home_win_prob"] = 1/(match_bets["overroundness"] * match_bets["1"])
match_bets["draw_prob"] = 1/(match_bets["overroundness"] * match_bets["X"])
match_bets["away_win_prob"] = 1/(match_bets["overroundness"] * match_bets["2"])
match_bets

,id,away_team,date,home_team,1,X,2,bet_model_outcome,overroundness,home_win_prob,draw_prob,away_win_prob
0,0,Saudi Arabia,2018-06-14,Russia,1.46,4.34,9.07,1.0,1.025600,0.667835,0.224663,0.107502
1,1,Uruguay,2018-06-15,Egypt,8.36,4.10,1.52,-1.0,1.021414,0.117109,0.238789,0.644102
2,2,IR Iran,2018-06-15,Morocco,2.23,2.98,4.16,1.0,1.024386,0.437756,0.327582,0.234662
3,3,Spain,2018-06-15,Portugal,4.38,3.28,2.04,-1.0,1.023385,0.223094,0.297911,0.478995
4,4,Australia,2018-06-16,France,1.27,7.22,10.39,1.0,1.022152,0.770337,0.135502,0.094161
5,5,Iceland,2018-06-16,Argentina,1.33,5.43,11.27,1.0,1.024773,0.733704,0.179710,0.086586
6,6,Denmark,2018-06-16,Peru,3.71,3.11,2.31,-1.0,1.023986,0.263228,0.314012,0.422760
7,7,Nigeria,2018-06-16,Croatia,1.68,3.78,6.14,1.0,1.022655,0.582052,0.258690,0.159258
8,8,Serbia,2018-06-17,Costa Rica,5.35,3.43,1.84,-1.0,1.021939,0.182903,0.285286,0.531811
9,9,Mexico,2018-06-17,Germany,1.49,4.56,7.20,1.0,1.029328,0.652018,0.213050,0.134932


In [51]:
data = pd.merge(match_template, match_bets, how='inner', left_on="id", right_on="id")
data.loc[data["bet_model_outcome"] == data["outcome"], "correct"] = 1
print("accuracy:", data["correct"].sum()/data.shape[0])

accuracy: 0.5625


In [69]:
y_pred = data["bet_model_outcome"].values
y_true = data["outcome"].values
odds = data[["1", "X", "2"]].values

bank = 64
for i in range(len(y_true)):
    bank -= 1
    predicted_outcome = y_pred[i]
    if y_true[i] == predicted_outcome:
        if predicted_outcome == 1:
            odd = odds[i, 0]
        elif predicted_outcome == 0:
            odd = odds[i, 1]
        else:
            odd = odds[i, 2]
        bank += odd
print(bank)

64.84
